In [1]:
# essential to initialize
import findspark
findspark.init()
# findspark.find()

In [2]:
# multiple ways to initialize sc
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext('local[16]','First App')

In [3]:
#spark = SparkSession(sc)

In [4]:
# reviews = spark.read.json('asnlib/publicdata/review.json')

In [5]:
# reviews.first()

In [6]:
# reviews_large = spark.read.json('asnlib/publicdata/review_large.json')

In [7]:
# reviews_large.first()

In [8]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')

In [9]:
# reviews_txt.collect()

In [10]:
reviews_txt.first()

'{"review_id": "-I5umRTkhw15RqpKMl_o1Q", "user_id": "-mA3-1mN4JIEkqOtdbNXCQ", "business_id": "mRUVMJkUGxrByzMQ2MuOpA", "stars": 1.0, "text": "Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don\'t even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!", "date": "2017-12-15 23:27:08"}'

In [11]:
import json

reviews_json = reviews_txt.map(lambda x: json.loads(x))
reviews_json.first()['date']

'2017-12-15 23:27:08'

In [12]:
reviews_json.count()

1151625

In [13]:
reviews_json.filter(lambda x: '2012' in x['date']).count()

63540

In [14]:
reviews_json.filter(lambda x: '2018' in x['date']).count()

202784

In [ ]:
reviews_json.map(lambda x: (x['date'][0:4], 1)).reduceByKey(lambda a,b: a+b).sortByKey(ascending=False).collect()

In [ ]:
reviews_json.map(lambda x: x['user_id']).distinct().count()

In [ ]:
user_review_count = reviews_json.map(lambda x: (x["user_id"], 1)).reduceByKey(lambda a, b: a + b)

In [ ]:
# user_review_count.collect()

In [ ]:
user_review_count.count() == reviews_json.map(lambda x: x['user_id']).distinct().count()

In [ ]:
user_review_count_sorted = user_review_count.sortBy(lambda x: x[1], ascending=False)

In [ ]:
# user_review_count_sorted.collect()

In [ ]:
user_review_count_sorted.take(5)

In [15]:
reviews_rdd = reviews_json.map(lambda x: (x['text']))
reviews_rdd.first()

"Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don't even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!"

In [16]:
punctuations = sc.parallelize(["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"])
stopwords = sc.textFile('asnlib/publicdata/stopwords')

In [17]:
punctuations.first()

'('

In [18]:
stopwords.first()

'i'

In [19]:
all_stopwords = stopwords.union(punctuations)

In [ ]:
all_stopwords.collect()

In [36]:
words_count_rdd = reviews_rdd.flatMap(lambda line: line.lower().split()).map(lambda x: (x.strip(),1)).reduceByKey(lambda a,b: a+b)


In [37]:
words_count_rdd.take(50)

[('even', 217970),
 ('they', 1021260),
 ('noticed', 18350),
 ('not', 751408),
 ('for', 1501662),
 ('from', 354127),
 ('sure', 100415),
 ('fresh', 76502),
 ('there', 398786),
 ('poland', 29),
 ('part', 36836),
 ('sell', 11430),
 ('buy', 28246),
 ('prices', 56896),
 ('wait', 98978),
 ('still', 111889),
 ('almost', 47540),
 ('restaurant,', 11570),
 ('which', 257237),
 ('before', 108940),
 ('filling,', 1356),
 ('well', 114339),
 ('fantastic!', 4819),
 ('flip', 1337),
 ('know', 122343),
 ('ever', 97181),
 ('elna', 2),
 ('took', 108965),
 ('waitress', 39763),
 ('card/', 7),
 ('away', 45502),
 ('toss', 852),
 ('phone', 31930),
 ('friday,', 1257),
 ("let's", 8384),
 ('fits', 1551),
 ('(1)', 467),
 ('literally', 19763),
 ('handle', 5798),
 ('truth', 1138),
 ('gotten', 12049),
 ('department', 4534),
 ('seen', 23784),
 ('places', 48991),
 ('kids', 30191),
 ('coupons.', 537),
 ('gem!', 1549),
 ('living', 9727),
 ('whatsoever', 663),
 ('pizza.', 12813)]

In [32]:
with open('asnlib/publicdata/stopwords') as f:
    f_stopwords = f.read()
    f.close()

In [ ]:
stopwords_list = [x.strip().lower() for x in f_stopwords.split()]

In [ ]:
stopwords_list.extend(["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"])

In [ ]:
words_count_rdd.filter(lambda x: x[0] not in stopwords_list).sortBy(lambda x: x[1], ascending=False).take(10)

In [34]:
stopwords_list.extend(["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"])

In [38]:
words_count_rdd.filter(lambda x: x[0] not in stopwords_list).sortBy(lambda x: x[1], ascending=False).take(10)

[('place', 436325),
 ('food', 429419),
 ('great', 395097),
 ('like', 387891),
 ('good', 376698),
 ('get', 375678),
 ('would', 347361),
 ('one', 344864),
 ("it's", 313474),
 ('service', 294992)]